In [1]:
from IPython.display import display,HTML,clear_output
import json

# Read explanation report

In [2]:
explanation_report_filepath = "/Users/salvatore/PycharmProjects/T-EBAnO-Express/outputs/20201128_bert_model_ag_news_subset_exp0/local_explanations_experiments/20201206_012332/local_explanations/local_explanation_report_4.json"




In [3]:
with open(explanation_report_filepath) as json_file:
    explanation_report = json.load(json_file)

# Parse explanation report

In [4]:
input_info = explanation_report["input_info"]
input_positions_tokens = input_info["positions_tokens"]
raw_text = input_info["raw_text"]
cleaned_text = input_info["cleaned_text"]
preprocessed_text = input_info["preprocessed_text"]

In [5]:
pos_local_explanations = []  # pos features without combinations
sen_local_explanations = []  # sen features without combinations
mlwe_local_explanations = []  # mlwe features without combinations
pos_local_explanations_comb = []  # All pos features (also combinations)
sen_local_explanations_comb = []  # All sen features (also combinations)
mlwe_local_explanations_comb = []  # All mlwe features (also combinations)

for le in explanation_report["local_explanations"]:
    if le["feature_type"] == "POS" and le["combination"] == 1:
        pos_local_explanations.append(le)
    if le["feature_type"] == "SEN" and le["combination"] == 1:
        sen_local_explanations.append(le)
    if le["feature_type"] == "MLWE" and le["combination"] == 1:
        mlwe_local_explanations.append(le)
    if le["feature_type"] == "POS":
        pos_local_explanations_comb.append(le)
    if le["feature_type"] == "SEN":  
        sen_local_explanations_comb.append(le)
    if le["feature_type"] == "MLWE":
        mlwe_local_explanations_comb.append(le)
        

In [6]:
pos_positions_tokens_score = {} # Dictionary for pos features with position as key and as value the tuple (token, nPIR)
sen_positions_tokens_score = {}#  Dictionary for sen features with position as key and as value the tuple (token, nPIR)
mlwe_positions_tokens_score = {} # Dictionary for mlwe features with position as key and as value the tuple (token, nPIR)

for le in pos_local_explanations:
    for position, token in le["positions_tokens"].items():
        pos_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))
        
for le in sen_local_explanations:
    for position, token in le["positions_tokens"].items():
        sen_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))

for le in mlwe_local_explanations:
    for position, token in le["positions_tokens"].items():
        mlwe_positions_tokens_score[position] = (token, round(le["nPIR_original_top_class"],4))
    

# Utility functions


In [7]:
def label_name(label):
    return ["World", "Sports", "Business", "Sci/Tech"][label]

def get_summary_string(position_token_score):
    positive_color = "background-color:#00FF00"
    negative_color = "background-color:#FF00FF"
    html_string = ""

    for position in sorted(position_token_score.keys(), key=lambda k: int(k)):
        token_score = position_token_score[position]
        if token_score[1] >= 0:
            positive_color = "background-color:rgba(124, 252, 0, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(positive_color,token_score[0])
        else:
            negative_color = "background-color:rgba(255, 99, 71, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(negative_color,token_score[0])
            
    return html_string

def get_feature_colored_html_string(input_positions_tokens, feature_positions_tokens):
    html_string = ""
    feature_color = "background-color:rgba(0, 255, 255, 1)"
    for position in range(len(input_positions_tokens)):
        if str(position) in feature_positions_tokens:
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(feature_color,input_positions_tokens[position])
        else:
            html_string = html_string + '<span><b>{}</b></span> '.format(input_positions_tokens[position])
    return html_string
    

def print_local_explanation(le, input_positions_tokens):
    if len(le["positions_tokens"]) == 0:
        return
    print("LocalExplanationId: {}, FeatureId: {}".format(le["local_explanation_id"], le["feature_id"]))
    print("feature type: ",le["feature_type"])
    print("feature description: ", le["feature_description"])
    print("feature (position,token): " ,le["positions_tokens"])
    print("Perturbed Text:  ",le["perturbed_text"])
    print("perturbed probabilities: ",le["perturbed_probabilities"])
    print("nPIR Original Class: " ,round(le["nPIR_original_top_class"],3))
    display(HTML(get_feature_colored_html_string(input_positions_tokens,le["positions_tokens"])))
    print("\n")
    print("\n")
    return

In [8]:
display(HTML('<h1>Input Text info</h1>'))
display(HTML('Raw Text:   <b>{} </b>'.format(raw_text)))
display(HTML('Clean Text:   <b>{} </b>'.format(cleaned_text)))
display(HTML('Preprocessed Text:   <b>{} </b>'.format(preprocessed_text)))

In [9]:
display(HTML('<h1>Multi-Layer Word Embedding Features Summary</h1>'))
display(HTML('Predicted Class: <b>{} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('Original Probability: <b>{} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(mlwe_positions_tokens_score)
        
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# MLWE Most influential feature

Feature with highest nPIR 

In [10]:
most_influnetial_mlwe_feature =max(mlwe_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_mlwe_feature,input_positions_tokens)

LocalExplanationId: 45, FeatureId: 2
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 1
feature (position,token):  {'11': ')', '12': '.', '41': ',', '49': '-', '51': '.', '54': "'", '55': 's', '64': ',', '66': '-', '72': ',', '84': "'", '97': '.', '0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '13': 'maccentral', '14': '-', '15': 'apple', '16': 'has', '17': 'improved', '18': 'the', '19': 'services', '20': 'offered', '21': 'to', '22': 'subscribers', '23': 'of', '24': '.', '25': 'mac', '26': '.', '27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '50': 'mail', '52': 'the', '53': 'service', '56': 'base', '57': 'online', '58'

<h2>Query Multi-Layer Word Embedding features</h2>

In [11]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_mlwe = -1
max_nPIR_mlwe = 1

for le in sorted(mlwe_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_mlwe and le["nPIR_original_top_class"] <= max_nPIR_mlwe:
        print_local_explanation(le, input_positions_tokens)
        

Original Probabilities: [0.00030222180066630244, 1.179747505375417e-05, 0.00014384924725163728, 0.9995421171188354]
Original label: 3



LocalExplanationId: 45, FeatureId: 2
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 1
feature (position,token):  {'11': ')', '12': '.', '41': ',', '49': '-', '51': '.', '54': "'", '55': 's', '64': ',', '66': '-', '72': ',', '84': "'", '97': '.', '0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '13': 'maccentral', '14': '-', '15': 'apple', '16': 'has', '17': 'improved', '18': 'the', '19': 'services', '20': 'offered', '21': 'to', '22': 'subscribers', '23': 'of', '24': '.', '25': 'mac', '26': '.', '27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '42': 'with', '43': 'a', '44': 'm





LocalExplanationId: 44, FeatureId: 1
feature type:  MLWE
feature description:  Cluster 1
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '13': 'maccentral', '14': '-', '15': 'apple', '16': 'has', '17': 'improved', '18': 'the', '19': 'services', '20': 'offered', '21': 'to', '22': 'subscribers', '23': 'of', '24': '.', '25': 'mac', '26': '.', '27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '50': 'mail', '52': 'the', '53': 'service', '56': 'base', '57': 'online', '58': 'storage', '59': 'has', '60': 'been', '61': 'increased', '62': 'to', '63': '250mb', '65': 'e', '67': 'mail', '68': 'service', '69': 'has', '70': 'been', '7





LocalExplanationId: 43, FeatureId: 0
feature type:  MLWE
feature description:  Cluster 0
feature (position,token):  {'11': ')', '12': '.', '41': ',', '49': '-', '51': '.', '54': "'", '55': 's', '64': ',', '66': '-', '72': ',', '84': "'", '97': '.'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral maccentral - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a basic. mac account was 100mb with a maximum of 15mb for e mail the service base online storage has been increased to 250mb e mail service has been enhanced and the cost of upgrading has been reduced.. mac s basic subscription price remains the same - - us per year
perturbed probabilities:  [2.4350423700525425e-05, 4.222318693791749e-06, 0.00011563351290533319, 0.9998557567596436]
nPIR Original Class:  -0.001


In [12]:
display(HTML('<h1>Parts-of-speech Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(pos_positions_tokens_score)
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# POS Most influential feature

In [13]:
most_influnetial_pos_feature =max(pos_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_pos_feature,input_positions_tokens)

LocalExplanationId: 7, FeatureId: 7
feature type:  POS
feature description:  Combination of Adjectives and Nouns
feature (position,token):  {'10': 'maccentral', '13': 'maccentral', '35': 'basic', '55': 's', '85': 's', '86': 'basic', '91': 'same', '1': 'mac', '4': 'storage', '5': 'capacity', '8': 'mail', '15': 'apple', '19': 'services', '22': 'subscribers', '25': 'mac', '30': 'amount', '32': 'storage', '37': 'mac', '38': 'account', '44': 'maximum', '48': 'e', '50': 'mail', '53': 'service', '56': 'base', '57': 'online', '58': 'storage', '67': 'mail', '68': 'service', '75': 'cost', '77': 'upgrading', '83': 'mac', '87': 'subscription', '88': 'price', '96': 'year'}
Perturbed Text:   . bumps up, improves ( ). - has improved the offered to of.. previously, the of for a. was 100mb, with a of 15mb for -. the ' has been increased to 250mb, e - has been enhanced, and the of has been reduced.. ' remains the - - us per.
perturbed probabilities:  [0.00029112669290043414, 0.00011168943456141278, 0.00

<h1>Query Parts-of-speech Features</h1>

In [14]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_pos = -1
max_nPIR_pos = 1

for le in sorted(pos_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_pos and le["nPIR_original_top_class"] <= max_nPIR_pos:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.00030222180066630244, 1.179747505375417e-05, 0.00014384924725163728, 0.9995421171188354]
Original label: 3



LocalExplanationId: 7, FeatureId: 7
feature type:  POS
feature description:  Combination of Adjectives and Nouns
feature (position,token):  {'10': 'maccentral', '13': 'maccentral', '35': 'basic', '55': 's', '85': 's', '86': 'basic', '91': 'same', '1': 'mac', '4': 'storage', '5': 'capacity', '8': 'mail', '15': 'apple', '19': 'services', '22': 'subscribers', '25': 'mac', '30': 'amount', '32': 'storage', '37': 'mac', '38': 'account', '44': 'maximum', '48': 'e', '50': 'mail', '53': 'service', '56': 'base', '57': 'online', '58': 'storage', '67': 'mail', '68': 'service', '75': 'cost', '77': 'upgrading', '83': 'mac', '87': 'subscription', '88': 'price', '96': 'year'}
Perturbed Text:   . bumps up, improves ( ). - has improved the offered to of.. previously, the of for a. was 100mb, with a of 15mb for -. the ' has been increased to 250mb, e - has been enhanced





LocalExplanationId: 17, FeatureId: 17
feature type:  POS
feature description:  Combination of Nouns and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'1': 'mac', '4': 'storage', '5': 'capacity', '8': 'mail', '15': 'apple', '19': 'services', '22': 'subscribers', '25': 'mac', '30': 'amount', '32': 'storage', '37': 'mac', '38': 'account', '44': 'maximum', '48': 'e', '50': 'mail', '53': 'service', '56': 'base', '57': 'online', '58': 'storage', '67': 'mail', '68': 'service', '75': 'cost', '77': 'upgrading', '83': 'mac', '87': 'subscription', '88': 'price', '96': 'year', '40': '100mb', '46': '15mb', '63': '250mb', '65': 'e'}
Perturbed Text:   . bumps up, improves ( maccentral ). maccentral - has improved the offered to of.. previously, the of for a basic. was, with a of for -. the ' s has been increased to, - has been enhanced, and the of has been reduced.. ' s basic remains the same - - us per.
perturbed probabilities:  [0.0001159535750048235, 8.880724635





LocalExplanationId: 3, FeatureId: 3
feature type:  POS
feature description:  Adverbs, Modals
feature (position,token):  {'27': 'previously'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). maccentral - apple has improved the services offered to subscribers of. mac., the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [0.00041087844874709845, 1.4577299225493334e-05, 0.00014049802848603576, 0.9994340538978577]
nPIR Original Class:  0.0






LocalExplanationId: 23, FeatureId: 23
feature type:  POS
feature description:  Combination of Adverbs, Modals and Interjections
feature (position,token):  {'27': 'previously'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). maccentral - apple has improved the services offered to subscribers of. mac., the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [0.00041087844874709845, 1.4577299225493334e-05, 0.00014049802848603576, 0.9994340538978577]
nPIR Original Class:  0.0






LocalExplanationId: 24, FeatureId: 24
feature type:  POS
feature description:  Combination of Adverbs, Modals and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'27': 'previously', '40': '100mb', '46': '15mb', '63': '250mb', '65': 'e'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). maccentral - apple has improved the services offered to subscribers of. mac., the amount of storage for a basic. mac account was, with a maximum of for e - mail. the service ' s base online storage has been increased to, - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [0.00031249303719960153, 1.2490080735005904e-05, 0.00019359696307219565, 0.9994814991950989]
nPIR Original Class:  0.0


...





LocalExplanationId: 10, FeatureId: 10
feature type:  POS
feature description:  Combination of Adjectives and Pronouns, Predeterminers, WH
feature (position,token):  {'10': 'maccentral', '13': 'maccentral', '35': 'basic', '55': 's', '85': 's', '86': 'basic', '91': 'same', '54': "'", '94': 'us'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( ). - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a. mac account was 100mb, with a maximum of 15mb for e - mail. the service base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' subscription price remains the - - per year.
perturbed probabilities:  [3.2226529583567753e-05, 3.6144508612778736e-06, 0.00022222546976990998, 0.9997419714927673]
nPIR Original Class:  -0.0






LocalExplanationId: 0, FeatureId: 0
feature type:  POS
feature description:  Adjectives
feature (position,token):  {'10': 'maccentral', '13': 'maccentral', '35': 'basic', '55': 's', '85': 's', '86': 'basic', '91': 'same'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( ). - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' subscription price remains the - - us per year.
perturbed probabilities:  [2.920807310147211e-05, 3.0988917387730908e-06, 0.00021043575543444604, 0.9997572302818298]
nPIR Original Class:  -0.0






LocalExplanationId: 11, FeatureId: 11
feature type:  POS
feature description:  Combination of Adjectives and Interjections
feature (position,token):  {'10': 'maccentral', '13': 'maccentral', '35': 'basic', '55': 's', '85': 's', '86': 'basic', '91': 'same'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( ). - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' subscription price remains the - - us per year.
perturbed probabilities:  [2.920807310147211e-05, 3.0988917387730908e-06, 0.00021043575543444604, 0.9997572302818298]
nPIR Original Class:  -0.0






LocalExplanationId: 9, FeatureId: 9
feature type:  POS
feature description:  Combination of Adjectives and Adverbs, Modals
feature (position,token):  {'10': 'maccentral', '13': 'maccentral', '35': 'basic', '55': 's', '85': 's', '86': 'basic', '91': 'same', '27': 'previously'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( ). - apple has improved the services offered to subscribers of. mac., the amount of storage for a. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' subscription price remains the - - us per year.
perturbed probabilities:  [3.501841638353653e-05, 3.431309551160666e-06, 0.0001956576743395999, 0.999765932559967]
nPIR Original Class:  -0.0






LocalExplanationId: 13, FeatureId: 13
feature type:  POS
feature description:  Combination of Nouns and Verbs
feature (position,token):  {'1': 'mac', '4': 'storage', '5': 'capacity', '8': 'mail', '15': 'apple', '19': 'services', '22': 'subscribers', '25': 'mac', '30': 'amount', '32': 'storage', '37': 'mac', '38': 'account', '44': 'maximum', '48': 'e', '50': 'mail', '53': 'service', '56': 'base', '57': 'online', '58': 'storage', '67': 'mail', '68': 'service', '75': 'cost', '77': 'upgrading', '83': 'mac', '87': 'subscription', '88': 'price', '96': 'year', '2': 'bumps', '3': 'up', '7': 'improves', '16': 'has', '17': 'improved', '20': 'offered', '39': 'was', '59': 'has', '60': 'been', '61': 'increased', '69': 'has', '70': 'been', '71': 'enhanced', '78': 'has', '79': 'been', '80': 'reduced', '89': 'remains'}
Perturbed Text:   ., ( maccentral ). maccentral - the to of.. previously, the of for a basic. 100mb, with a of 15mb for -. the ' s to 250mb, e -, and the of.. ' s basic the same - -

In [15]:
display(HTML('<h1>Sentences Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_sen_string = get_summary_string(sen_positions_tokens_score)
        
display(HTML(html_sen_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# SEN Most influential feature

In [16]:
most_influnetial_sen_feature =max(sen_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_sen_feature,input_positions_tokens)

LocalExplanationId: 34, FeatureId: 6
feature type:  SEN
feature description:  Combination of 1° Sentence and 3° Sentence
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '11': ')', '12': '.', '27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '41': ',', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '49': '-', '50': 'mail', '51': '.'}
Perturbed Text:   maccentral - apple has improved the services offered to subscribers of. mac. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [0.000106

<h1> Query Sentences Features</h1>

In [17]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_sen = -1
max_nPIR_sen = 1

for le in sorted(sen_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_sen and le["nPIR_original_top_class"] <= max_nPIR_sen:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.00030222180066630244, 1.179747505375417e-05, 0.00014384924725163728, 0.9995421171188354]
Original label: 3



LocalExplanationId: 34, FeatureId: 6
feature type:  SEN
feature description:  Combination of 1° Sentence and 3° Sentence
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '11': ')', '12': '.', '27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '41': ',', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '49': '-', '50': 'mail', '51': '.'}
Perturbed Text:   maccentral - apple has improved the services offered to subscribers of. mac. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the co





LocalExplanationId: 37, FeatureId: 9
feature type:  SEN
feature description:  Combination of 2° Sentence and 3° Sentence
feature (position,token):  {'13': 'maccentral', '14': '-', '15': 'apple', '16': 'has', '17': 'improved', '18': 'the', '19': 'services', '20': 'offered', '21': 'to', '22': 'subscribers', '23': 'of', '24': '.', '25': 'mac', '26': '.', '27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '41': ',', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '49': '-', '50': 'mail', '51': '.'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed pr





LocalExplanationId: 33, FeatureId: 5
feature type:  SEN
feature description:  Combination of 1° Sentence and 2° Sentence
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '11': ')', '12': '.', '13': 'maccentral', '14': '-', '15': 'apple', '16': 'has', '17': 'improved', '18': 'the', '19': 'services', '20': 'offered', '21': 'to', '22': 'subscribers', '23': 'of', '24': '.', '25': 'mac', '26': '.'}
Perturbed Text:   previously, the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [6.192366709001362e-05, 6.71190127832233e-06, 0.000864101224578917, 0.9990672469139099]
nPIR Original Clas





LocalExplanationId: 41, FeatureId: 13
feature type:  SEN
feature description:  Combination of 3° Sentence and 5° Sentence
feature (position,token):  {'27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '41': ',', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '49': '-', '50': 'mail', '51': '.', '82': '.', '83': 'mac', '84': "'", '85': 's', '86': 'basic', '87': 'subscription', '88': 'price', '89': 'remains', '90': 'the', '91': 'same', '92': '-', '93': '-', '94': 'us', '95': 'per', '96': 'year', '97': '.'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). maccentral - apple has improved the services offered to subscribers of. mac. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been re





LocalExplanationId: 28, FeatureId: 0
feature type:  SEN
feature description:  1° Sentence
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '11': ')', '12': '.'}
Perturbed Text:   maccentral - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [5.6347074860241264e-05, 4.781167717737844e-06, 0.0007852500420995057, 0.9991536140441895]
nPIR Original Class:  0.001


...





LocalExplanationId: 35, FeatureId: 7
feature type:  SEN
feature description:  Combination of 1° Sentence and 4° Sentence
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '11': ')', '12': '.', '52': 'the', '53': 'service', '54': "'", '55': 's', '56': 'base', '57': 'online', '58': 'storage', '59': 'has', '60': 'been', '61': 'increased', '62': 'to', '63': '250mb', '64': ',', '65': 'e', '66': '-', '67': 'mail', '68': 'service', '69': 'has', '70': 'been', '71': 'enhanced', '72': ',', '73': 'and', '74': 'the', '75': 'cost', '76': 'of', '77': 'upgrading', '78': 'has', '79': 'been', '80': 'reduced', '81': '.'}
Perturbed Text:   maccentral - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail.. mac ' s basic subscription price remains the same - - 





LocalExplanationId: 38, FeatureId: 10
feature type:  SEN
feature description:  Combination of 2° Sentence and 4° Sentence
feature (position,token):  {'13': 'maccentral', '14': '-', '15': 'apple', '16': 'has', '17': 'improved', '18': 'the', '19': 'services', '20': 'offered', '21': 'to', '22': 'subscribers', '23': 'of', '24': '.', '25': 'mac', '26': '.', '52': 'the', '53': 'service', '54': "'", '55': 's', '56': 'base', '57': 'online', '58': 'storage', '59': 'has', '60': 'been', '61': 'increased', '62': 'to', '63': '250mb', '64': ',', '65': 'e', '66': '-', '67': 'mail', '68': 'service', '69': 'has', '70': 'been', '71': 'enhanced', '72': ',', '73': 'and', '74': 'the', '75': 'cost', '76': 'of', '77': 'upgrading', '78': 'has', '79': 'been', '80': 'reduced', '81': '.'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). previously, the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail.. mac ' s basic subscription price r





LocalExplanationId: 36, FeatureId: 8
feature type:  SEN
feature description:  Combination of 1° Sentence and 5° Sentence
feature (position,token):  {'0': '.', '1': 'mac', '2': 'bumps', '3': 'up', '4': 'storage', '5': 'capacity', '6': ',', '7': 'improves', '8': 'mail', '9': '(', '10': 'maccentral', '11': ')', '12': '.', '82': '.', '83': 'mac', '84': "'", '85': 's', '86': 'basic', '87': 'subscription', '88': 'price', '89': 'remains', '90': 'the', '91': 'same', '92': '-', '93': '-', '94': 'us', '95': 'per', '96': 'year', '97': '.'}
Perturbed Text:   maccentral - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail. the service ' s base online storage has been increased to 250mb, e - mail service has been enhanced, and the cost of upgrading has been reduced.
perturbed probabilities:  [0.00016763078747317195, 8.756035640544724e-06, 0.000372912734746933, 0.9994507431983948





LocalExplanationId: 40, FeatureId: 12
feature type:  SEN
feature description:  Combination of 3° Sentence and 4° Sentence
feature (position,token):  {'27': 'previously', '28': ',', '29': 'the', '30': 'amount', '31': 'of', '32': 'storage', '33': 'for', '34': 'a', '35': 'basic', '36': '.', '37': 'mac', '38': 'account', '39': 'was', '40': '100mb', '41': ',', '42': 'with', '43': 'a', '44': 'maximum', '45': 'of', '46': '15mb', '47': 'for', '48': 'e', '49': '-', '50': 'mail', '51': '.', '52': 'the', '53': 'service', '54': "'", '55': 's', '56': 'base', '57': 'online', '58': 'storage', '59': 'has', '60': 'been', '61': 'increased', '62': 'to', '63': '250mb', '64': ',', '65': 'e', '66': '-', '67': 'mail', '68': 'service', '69': 'has', '70': 'been', '71': 'enhanced', '72': ',', '73': 'and', '74': 'the', '75': 'cost', '76': 'of', '77': 'upgrading', '78': 'has', '79': 'been', '80': 'reduced', '81': '.'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). maccentral 





LocalExplanationId: 31, FeatureId: 3
feature type:  SEN
feature description:  4° Sentence
feature (position,token):  {'52': 'the', '53': 'service', '54': "'", '55': 's', '56': 'base', '57': 'online', '58': 'storage', '59': 'has', '60': 'been', '61': 'increased', '62': 'to', '63': '250mb', '64': ',', '65': 'e', '66': '-', '67': 'mail', '68': 'service', '69': 'has', '70': 'been', '71': 'enhanced', '72': ',', '73': 'and', '74': 'the', '75': 'cost', '76': 'of', '77': 'upgrading', '78': 'has', '79': 'been', '80': 'reduced', '81': '.'}
Perturbed Text:   . mac bumps up storage capacity, improves mail ( maccentral ). maccentral - apple has improved the services offered to subscribers of. mac. previously, the amount of storage for a basic. mac account was 100mb, with a maximum of 15mb for e - mail.. mac ' s basic subscription price remains the same - - us per year.
perturbed probabilities:  [0.0002799734938889742, 1.4762285900360439e-05, 8.554979285690933e-05, 0.999619722366333]
nPIR Origin